In [ ]:
### import os
import os
import sys

## CHANGE THIS
dir2 = os.path.abspath("/volume/Orion/orion")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
import importlib

importlib.util.find_spec("orion")
# import torch
import csv

import numpy as np
import pandas as pd
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 250)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 300)
pd.options.mode.chained_assignment = None  # default='warn'

def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    df_stat_val = pd.DataFrame({
        "Name": df.columns,
        "Null": df.isnull().sum(),
        "Unique": df.nunique(),
        "Dtypes": df.dtypes
    })

    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
import pandas as pd
from orion.utils import video_training_and_eval, get_mean_and_std
from orion import datasets
import yaml
import torch
import wandb
from orion.utils.plot import (
    initialize_classification_metrics,
    initialize_regression_metrics
)

def run_inference(checkpoints_folder):

    with open('config/config_regression_swin3d.yaml') as file:
        config = yaml.safe_load(file)

    config['model_path'] = checkpoints_folder + '/best.pt'
    config['wandb_id'] = '4iivz6og'  # change as needed
    config['resume'] = True
    config['debug'] = False
    config["output"] = checkpoints_folder + '/'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if (config['task'] == 'classification'):
        num_classes = 12 if config["num_classes"] <= 1 else config["num_classes"]
        metrics = {
            "train": initialize_classification_metrics(num_classes, device),
            "val": initialize_classification_metrics(num_classes, device),
            "test": initialize_classification_metrics(num_classes, device),
        }
    else:
        metrics = {
            "train": initialize_regression_metrics(device),
            "val": initialize_regression_metrics(device),
            "test": initialize_regression_metrics(device),
        }

    best_metrics = {
        "val": {
            "best_loss": float("inf"),
            "best_auc": -float("inf"),  # for classification
            "best_mae": float("inf"),  # for regression
            "best_rmse": float("inf"),  # for regression
        },
        "test": {
            "best_loss": float("inf"),
            "best_auc": -float("inf"),  # for classification
            "best_mae": float("inf"),  # for regression
            "best_rmse": float("inf"),  # for regression
        },
    }
    wandb.init(
                entity=config["entity"],
                project=config["project"],
                config=config,
                name=config["project"],  
                resume=config.get("resume", False),
                id=config.get("wandb_id", None),
            )
    (
        predictions
    ) = video_training_and_eval.perform_inference(
        config=config,
        split='test',
        metrics=metrics,
        best_metrics=best_metrics,
    )

    # return predictions dataframe as per your need
    return df_predictions_inference

# Plot predictions for sanity check

In [ ]:
# os.environ['NCCL_P2P_DISABLE'] = '1'

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import torchvision.models.video

## CHANGE THIS
dir2 = os.path.abspath("/volume/Orion/orion")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
import importlib

importlib.util.find_spec("orion")

import orion.models
from orion.models import get_fmodel, pytorchvideo_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = pytorchvideo_model("swin3d_s", 1, "regression")
map_location = {"cuda:0"}



import torch
checkpoint = torch.load('../final_models/swin3d_s_5_32_2_RAdam_new_20231227-115517/best.pt')
# print("Model ddp", checkpoint)

model_state_dict = checkpoint["state_dict"]
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
model_state_dict, "_orig_mod.module."
)

model.load_state_dict(model_state_dict)
model.to(device)

optimizer_state = checkpoint.get("optimizer")
scheduler_state = checkpoint.get("scheduler")
epoch = checkpoint.get("epoch", 0)
bestLoss = checkpoint.get("best_loss", float("inf"))
other_metrics = {
    k: v
    for k, v in checkpoint.items()
    if k
    not in ["model_state_dict", "optimizer_state_dict", "scheduler_state_dict", "epoch"]
}